In [1]:
# ========================================
# English-Telugu Translation with mBART
# ========================================
# This notebook fine-tunes mBART for English↔Telugu translation

# ========================================
# CELL 1: Install Dependencies
# ========================================
!pip install -q transformers datasets pandas torch accelerate huggingface_hub

In [2]:
# ========================================
# CELL 2: Import Libraries
# ========================================
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import os

# Disable W&B if you don't want to use it
os.environ["WANDB_DISABLED"] = "true"

print("✅ Libraries imported successfully!")
print(f"🖥️  Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

✅ Libraries imported successfully!
🖥️  Using device: GPU


In [3]:
# ========================================
# CELL 3: Download and Prepare Dataset
# ========================================
def prepare_telugu_dataset():
    """
    Downloads Telugu-English parallel corpus and prepares train/val splits
    """
    print("📥 Downloading dataset...")

    # Load the Telugu translation dataset
    df = pd.read_parquet(
        "hf://datasets/Shreya3095/TeluguTranslator/data/train-00000-of-00001.parquet"
    )

    # Rename columns for clarity
    df = df.rename(columns={'question': 'english', 'answer': 'telugu'})

    # Clean the data
    print("🧹 Cleaning data...")
    df = df.dropna()
    df['english'] = df['english'].astype(str).str.strip()
    df['telugu'] = df['telugu'].astype(str).str.strip()

    # Filter by length (remove very short/long sentences)
    df = df[(df['english'].str.len() > 3) & (df['english'].str.len() < 200)]
    df = df[(df['telugu'].str.len() > 3) & (df['telugu'].str.len() < 300)]

    print(f"📊 Dataset size after cleaning: {len(df)} samples")

    # Split into train (90%) and validation (10%)
    from sklearn.model_selection import train_test_split
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

    print(f"📚 Train samples: {len(train_df)}")
    print(f"📖 Validation samples: {len(val_df)}")

    # Save to disk (optional, for inspection)
    os.makedirs('data', exist_ok=True)
    train_df.to_csv('data/train.csv', index=False)
    val_df.to_csv('data/val.csv', index=False)

    # Convert to Hugging Face Dataset format
    train_dataset = Dataset.from_pandas(train_df[['english', 'telugu']])
    val_dataset = Dataset.from_pandas(val_df[['english', 'telugu']])

    return train_dataset, val_dataset

# Run the preparation
train_dataset, val_dataset = prepare_telugu_dataset()

# Show a sample
print("\n🔍 Sample data:")
print(train_dataset[0])

📥 Downloading dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🧹 Cleaning data...
📊 Dataset size after cleaning: 1101 samples
📚 Train samples: 990
📖 Validation samples: 111

🔍 Sample data:
{'english': "It's time to get up.", 'telugu': 'ఇది లేవడానికి సమయం.', '__index_level_0__': 758}


In [4]:
# ========================================
# CELL 4: Load Pre-trained mBART Model
# ========================================
print("\n🤖 Loading pre-trained mBART model...")

# Model name from Hugging Face
model_name = "facebook/mbart-large-50-many-to-many-mmt"

# Load tokenizer and model
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set source and target languages
tokenizer.src_lang = "en_XX"  # English
tokenizer.tgt_lang = "te_IN"  # Telugu

print("✅ Model loaded successfully!")
print(f"📦 Model size: {sum(p.numel() for p in model.parameters()):,} parameters")


🤖 Loading pre-trained mBART model...


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

✅ Model loaded successfully!
📦 Model size: 610,879,488 parameters


In [5]:
# ========================================
# CELL 5: Preprocess Data for Training
# ========================================
def preprocess_function(examples):
    """
    Tokenizes English and Telugu text for training
    """
    # Tokenize English (source)
    inputs = tokenizer(
        examples["english"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # Tokenize Telugu (target)
    targets = tokenizer(
        examples["telugu"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # Labels for the model to learn from
    inputs["labels"] = targets["input_ids"]

    return inputs

print("🔄 Tokenizing datasets...")
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["english", "telugu"]
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["english", "telugu"]
)

print("✅ Tokenization complete!")

🔄 Tokenizing datasets...


Map:   0%|          | 0/990 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

✅ Tokenization complete!


In [6]:
# ========================================
# CELL 6: Setup Training Configuration
# ========================================
print("\n⚙️ Configuring training parameters...")

# Data collator - handles batching
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",                    # Where to save checkpoints
    eval_strategy="epoch",                      # Evaluate after each epoch
    save_strategy="epoch",                      # Save after each epoch
    learning_rate=5e-5,                         # Learning rate (standard for fine-tuning)
    per_device_train_batch_size=8,              # Batch size per GPU
    per_device_eval_batch_size=8,
    num_train_epochs=10,                        # Number of training epochs
    weight_decay=0.01,                          # Regularization
    save_total_limit=3,                         # Keep only last 3 checkpoints
    predict_with_generate=True,                 # Use generation for evaluation
    fp16=torch.cuda.is_available(),             # Use mixed precision if GPU available
    logging_dir="./logs",
    logging_steps=100,                          # Log every 100 steps
    load_best_model_at_end=True,               # Load best model at end
    metric_for_best_model="loss",               # Use loss to determine best model
    push_to_hub=False,                          # Don't auto-push to HF Hub
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("✅ Trainer configured!")


⚙️ Configuring training parameters...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1175056114.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


✅ Trainer configured!


In [7]:
# ========================================
# CELL 7: Train the Model
# ========================================
print("\n🚀 Starting training...")
print("⏱️  This will take ~20-30 minutes depending on your GPU\n")

# Start training
trainer.train()

print("\n🎉 Training complete!")


🚀 Starting training...
⏱️  This will take ~20-30 minutes depending on your GPU



Epoch,Training Loss,Validation Loss
1,2.666800,0.159782
2,0.123000,0.140520
3,0.063500,0.132383
4,0.034400,0.134058
5,0.010200,0.132840
6,0.009500,0.134899
7,0.006900,0.133653
8,0.004400,0.134699
9,0.003600,0.135549
10,0.003500,0.135425


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



🎉 Training complete!


In [8]:
# ========================================
# CELL 8: Save the Fine-tuned Model
# ========================================
print("\n💾 Saving model...")

# Save to local directory
output_dir = "./mbart-finetuned-en-te"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Model saved to: {output_dir}")


💾 Saving model...
✅ Model saved to: ./mbart-finetuned-en-te


In [10]:
# ========================================
# CELL 9: Test the Model
# ========================================
def translate(text, model, tokenizer, device="cuda", max_len=128):
    """
    Translates English text to Telugu
    """
    # Tokenize input
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=max_len,
        truncation=True
    ).to(device)

    # Generate translation
    generated_ids = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["te_IN"],
        max_length=max_len,
        num_beams=5,  # Beam search for better quality
        early_stopping=True
    )

    # Decode the output
    translation = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return translation

# Load the fine-tuned model for inference
print("\n🔮 Testing translations...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Test sentences
test_sentences = [
    "Hello, how are you?",
    "Good morning everyone",
    "I will go to the temple daily",
    "What is your name?",
    "Thank you very much",
    "I went to school yesterday"
]

print("\n" + "="*60)
print("TRANSLATION RESULTS")
print("="*60)

for english_text in test_sentences:
    telugu_text = translate(english_text, model, tokenizer, device)
    print(f"\n📝 EN: {english_text}")
    print(f"🔤 TE: {telugu_text}")
    print("-" * 60)


🔮 Testing translations...

TRANSLATION RESULTS

📝 EN: Hello, how are you?
🔤 TE: హలో, ఎలా ఉన్నావు?
------------------------------------------------------------

📝 EN: Good morning everyone
🔤 TE: అందరూ ఉదయం గుడ్ మార్నింగ్
------------------------------------------------------------

📝 EN: I will go to the temple daily
🔤 TE: నేను ప్రతి రోజు ఆలయంకి వెళ్తాను
------------------------------------------------------------

📝 EN: What is your name?
🔤 TE: మీ పేరు ఏమిటి?
------------------------------------------------------------

📝 EN: Thank you very much
🔤 TE: చాలా ధన్యవాదాలు
------------------------------------------------------------

📝 EN: I went to school yesterday
🔤 TE: నేను నిన్న బడికి వెళ్ళాను
------------------------------------------------------------


In [ ]:
# ========================================
# CELL 10: (Optional) Push to Hugging Face Hub
# ========================================
print("\n📤 To upload your model to Hugging Face Hub:")
print("1. Run: !huggingface-cli login")
print("2. Enter your token from https://huggingface.co/settings/tokens")
print("3. Uncomment and run the code below:\n")

# Uncomment these lines after logging in:
# from huggingface_hub import HfApi
#
# # Replace with your username and desired model name
# repo_id = "your-username/mbart-en-te"
#
# model.push_to_hub(repo_id)
# tokenizer.push_to_hub(repo_id)
#
# print(f"✅ Model uploaded to: https://huggingface.co/{repo_id}")

In [11]:
# ========================================
# CELL 11: Compare with Base Model
# ========================================
print("\n📊 Comparing Fine-tuned vs Base Model...")

# Load base model for comparison
base_model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
base_tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
base_tokenizer.src_lang = "en_XX"
base_tokenizer.tgt_lang = "te_IN"

test_text = "I love learning new languages"

print("\n" + "="*60)
print("BASE MODEL vs FINE-TUNED MODEL")
print("="*60)
print(f"\n📝 Input: {test_text}")
print(f"\n🔹 Base Model Output:")
base_output = translate(test_text, base_model, base_tokenizer, device)
print(f"   {base_output}")

print(f"\n🔸 Fine-tuned Model Output:")
finetuned_output = translate(test_text, model, tokenizer, device)
print(f"   {finetuned_output}")
print("="*60)

print("\n✨ Notice the difference in quality!")
print("Fine-tuning adapts the model to your specific data distribution.")


📊 Comparing Fine-tuned vs Base Model...

BASE MODEL vs FINE-TUNED MODEL

📝 Input: I love learning new languages

🔹 Base Model Output:
   నేను కొత్త భాషలు తెలుసుకోవడానికి ప్రేమ

🔸 Fine-tuned Model Output:
   నాకు కొత్త భాషలు నేర్చుకోవడానికి చాలా ఇష్టం

✨ Notice the difference in quality!
Fine-tuning adapts the model to your specific data distribution.


In [12]:
# ========================================
# CELL 12: Create a Simple Translation Function
# ========================================
def create_translator(model_path="./mbart-finetuned-en-te"):
    """
    Creates a ready-to-use translation function
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = MBartForConditionalGeneration.from_pretrained(model_path).to(device)
    tokenizer = MBart50TokenizerFast.from_pretrained(model_path)
    tokenizer.src_lang = "en_XX"

    def translate_text(text, target_lang="te_IN"):
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id[target_lang],
            num_beams=5
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return translate_text

# Create translator function
translator = create_translator()

# Test it
print("\n🎯 Quick Translation Test:")
print(translator("How are you today?"))

print("\n" + "="*60)
print("🎓 TUTORIAL COMPLETE!")
print("="*60)
print("\n📚 What you learned:")
print("  ✓ How to fine-tune mBART for translation")
print("  ✓ Data preprocessing for sequence-to-sequence tasks")
print("  ✓ Using Hugging Face Trainer API")
print("  ✓ Evaluating translation quality")
print("  ✓ Deploying models to Hugging Face Hub")
print("\n💡 Next steps:")
print("  • Try with more data for better results")
print("  • Experiment with different hyperparameters")
print("  • Add evaluation metrics (BLEU, METEOR)")
print("  • Deploy as an API using FastAPI")
print("\nHappy learning! 🚀")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🎯 Quick Translation Test:
ఈ రోజు మీరు ఎలా ఉన్నారు?

🎓 TUTORIAL COMPLETE!

📚 What you learned:
  ✓ How to fine-tune mBART for translation
  ✓ Data preprocessing for sequence-to-sequence tasks
  ✓ Using Hugging Face Trainer API
  ✓ Evaluating translation quality
  ✓ Deploying models to Hugging Face Hub

💡 Next steps:
  • Try with more data for better results
  • Experiment with different hyperparameters
  • Add evaluation metrics (BLEU, METEOR)
  • Deploy as an API using FastAPI

Happy learning! 🚀
